In [1]:
from datetime import datetime
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm as tqdm_module
import requests
import re
import json
import pbmd_tools as pbmd

In [2]:
db = 'pubmed'
domain = 'https://www.ncbi.nlm.nih.gov/entrez/eutils'
nresults = 100
query = '"github.com"[Title/Abstract] NOT "github.com"[Title]'
retmode = 'json'

In [3]:
queryLinkSearch = f'{domain}/esearch.fcgi?db={db}&retmax={nresults}&retmode={retmode}&term={query}'
response = requests.get(queryLinkSearch)
pubmedJson = response.json()
results = []
for paperId in pubmedJson["esearchresult"]["idlist"]:
    queryLinkSummary = f'{domain}/esummary.fcgi?db={db}&id={paperId}&retmode={retmode}'
    results.append({'paperId': paperId, 'metadata': requests.get(queryLinkSummary).json()})

In [4]:
df1 = pd.DataFrame.from_records(results)
df1["result"] = df1["metadata"].apply(lambda x: x["result"])
df1["PMID"] = df1["paperId"]
df1["PubDate"] = [d[k]["sortpubdate"] for d in df1["result"] for k in d.keys() if k != "uids"]
df1["PubDate"] = df1["PubDate"].apply(lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M").strftime("%Y-%m-%d") if x else None)
df1["DOI"] = [d[k]["articleids"][i]["value"] for d in df1["result"] for k in d.keys() if k != "uids" for i in range(len(d[k]["articleids"])) if d[k]["articleids"][i]["idtype"] == "doi"] 
df1["Journal"] = [d[k]["fulljournalname"] for d in df1["result"] for k in d.keys() if k != "uids"]
df1["Title"] = [d[k]["title"] for d in df1["result"] for k in d.keys() if k != "uids"]
df1 = df1.drop(["paperId", "result", "metadata"], axis=1)
df1

,PMID,PubDate,DOI,Journal,Title
0,36760999,2023-01-24,10.3389/fgene.2023.1082032,Frontiers in genetics,Identification of functional gene modules by i...
1,36759692,2023-02-09,10.1038/s41598-023-29320-6,Scientific reports,Deep consistency-preserving hash auto-encoders...
2,36759336,2023-02-09,10.1093/bib/bbad044,Briefings in bioinformatics,HiConfidence: a novel approach uncovering the ...
3,36756726,2023-02-09,10.1111/1755-0998.13765,Molecular ecology resources,Long amplicon HiFi sequencing for mitochondria...
4,36756173,2022-10-23,10.1016/j.csbj.2022.10.016,Computational and structural biotechnology jou...,Multi-task deep autoencoder to predict Alzheim...
...,...,...,...,...,...
95,36711471,2023-01-20,10.1101/2023.01.17.524477,bioRxiv : the preprint server for biology,Combining protein sequences and structures wit...
96,36710930,2023-01-13,10.3389/fnmol.2022.1037565,Frontiers in molecular neuroscience,Inferring cell developmental stage-specific ln...
97,36710872,2023-01-23,10.7717/peerj.14706,PeerJ,A clustering method for small scRNA-seq data b...
98,36709790,2023-01-26,10.1016/j.ymeth.2023.01.006,"Methods (San Diego, Calif.)",Predicting latent lncRNA and cancer metastatic...


In [5]:
def get_abstract(PMID):
    queryLinkSearch = f'{domain}/efetch.fcgi?db={db}&id={PMID}&retmax={nresults}&retmode={retmode}&rettype=abstract'
    response = requests.get(queryLinkSearch)
    return response.text

In [ ]:
rate_limit = 3
request_count = 0

for id_iter in df1["PMID"]:  
    
    if request_count >= rate_limit:
        
        time.sleep(1)
        request_count = 0
        
    df1.loc[df1["PMID"] == id_iter,"Abstract"] = get_abstract(id_iter)
    
    request_count += 1
df1

,PMID,PubDate,DOI,Journal,Title,Abstract
0,36760999,2023-01-24,10.3389/fgene.2023.1082032,Frontiers in genetics,Identification of functional gene modules by i...,Front Genet. 2023 Jan 24;14:1082032. doi: 10.3...
1,36759692,2023-02-09,10.1038/s41598-023-29320-6,Scientific reports,Deep consistency-preserving hash auto-encoders...,Sci Rep. 2023 Feb 9;13(1):2316. doi: 10.1038/s...
2,36759336,2023-02-09,10.1093/bib/bbad044,Briefings in bioinformatics,HiConfidence: a novel approach uncovering the ...,Brief Bioinform. 2023 Feb 9:bbad044. doi: 10.1...
3,36756726,2023-02-09,10.1111/1755-0998.13765,Molecular ecology resources,Long amplicon HiFi sequencing for mitochondria...,Mol Ecol Resour. 2023 Feb 9. doi: 10.1111/1755...
4,36756173,2022-10-23,10.1016/j.csbj.2022.10.016,Computational and structural biotechnology jou...,Multi-task deep autoencoder to predict Alzheim...,Comput Struct Biotechnol J. 2022 Oct 23;20:576...
...,...,...,...,...,...,...
95,36711471,2023-01-20,10.1101/2023.01.17.524477,bioRxiv : the preprint server for biology,Combining protein sequences and structures wit...,bioRxiv. 2023 Jan 20:2023.01.17.524477. doi: 1...
96,36710930,2023-01-13,10.3389/fnmol.2022.1037565,Frontiers in molecular neuroscience,Inferring cell developmental stage-specific ln...,Front Mol Neurosci. 2023 Jan 13;15:1037565. do...
97,36710872,2023-01-23,10.7717/peerj.14706,PeerJ,A clustering method for small scRNA-seq data b...,PeerJ. 2023 Jan 23;11:e14706. doi: 10.7717/pee...
98,36709790,2023-01-26,10.1016/j.ymeth.2023.01.006,"Methods (San Diego, Calif.)",Predicting latent lncRNA and cancer metastatic...,Methods. 2023 Jan 26;211:1-9. doi: 10.1016/j.y...


In [ ]:
def get_link(df, PMID):
    
    regex = ["https:\/[^ ]*[./]github.com[^\n .,)]*"]
    links_with_point = ''
    
    for rgx in regex:
        if (links_with_point == '') and re.search(rgx, str(df.loc[df["PMID"] == PMID, "Abstract"].values[0]), re.IGNORECASE):
                links_with_point = re.findall(rgx, str(df.loc[df["PMID"] == PMID, "Abstract"].values[0]), re.IGNORECASE)
    links = []
    for link in links_with_point :
        links.append(link) 
        
    return links

In [ ]:
df1

,PMID,PubDate,DOI,Journal,Title,Abstract
0,36760999,2023-01-24,10.3389/fgene.2023.1082032,Frontiers in genetics,Identification of functional gene modules by i...,Front Genet. 2023 Jan 24;14:1082032. doi: 10.3...
1,36759692,2023-02-09,10.1038/s41598-023-29320-6,Scientific reports,Deep consistency-preserving hash auto-encoders...,Sci Rep. 2023 Feb 9;13(1):2316. doi: 10.1038/s...
2,36759336,2023-02-09,10.1093/bib/bbad044,Briefings in bioinformatics,HiConfidence: a novel approach uncovering the ...,Brief Bioinform. 2023 Feb 9:bbad044. doi: 10.1...
3,36756726,2023-02-09,10.1111/1755-0998.13765,Molecular ecology resources,Long amplicon HiFi sequencing for mitochondria...,Mol Ecol Resour. 2023 Feb 9. doi: 10.1111/1755...
4,36756173,2022-10-23,10.1016/j.csbj.2022.10.016,Computational and structural biotechnology jou...,Multi-task deep autoencoder to predict Alzheim...,Comput Struct Biotechnol J. 2022 Oct 23;20:576...
...,...,...,...,...,...,...
95,36711471,2023-01-20,10.1101/2023.01.17.524477,bioRxiv : the preprint server for biology,Combining protein sequences and structures wit...,bioRxiv. 2023 Jan 20:2023.01.17.524477. doi: 1...
96,36710930,2023-01-13,10.3389/fnmol.2022.1037565,Frontiers in molecular neuroscience,Inferring cell developmental stage-specific ln...,Front Mol Neurosci. 2023 Jan 13;15:1037565. do...
97,36710872,2023-01-23,10.7717/peerj.14706,PeerJ,A clustering method for small scRNA-seq data b...,PeerJ. 2023 Jan 23;11:e14706. doi: 10.7717/pee...
98,36709790,2023-01-26,10.1016/j.ymeth.2023.01.006,"Methods (San Diego, Calif.)",Predicting latent lncRNA and cancer metastatic...,Methods. 2023 Jan 26;211:1-9. doi: 10.1016/j.y...


In [ ]:
def clean_link(link_array):
    links = []
    for link in link_array:
        if link != "":
            if not link.startswith("https://"):
                link = "https://" + link
            if link[-2] == ")":
                link = link[:-2]
            if link[-1] == ")" or link[-1] == "." or link[-1] == "," or link[-1] == ",":
                link = link[:-1]
        links.append(link)
        
    return links

In [ ]:
def get_phrase_with_link(df, PMID):
    
    regex = ["\.[^.]*[./]github[^ ]* [^.]*[^ ]*\.", "\.[^.]*[./]github[^ ]*", "[^.]*github[^ ]*", "[a-zA-Z0-9 .,/:\'\"!?]{101}github[^ ]*[a-zA-Z0-9 .,/:\'\"!?()]{100}"]
    phrase_with_point = ''
    
    for rgx in regex:
        if (phrase_with_point == "") and re.search(rgx, str(df.loc[df["PMID"] == PMID, "Abstract"].values[0]), re.IGNORECASE):
            phrase_with_point = re.findall(rgx, str(df.loc[df["PMID"] == PMID, "Abstract"].values[0]), re.IGNORECASE)
    phrases = []
    for phrase in phrase_with_point :
        phrase = phrase[2:]
        phrases.append(phrase)
        
    return ' '.join(phrases)

In [ ]:
for id_iter in df1["PMID"]:
    
    df1.loc[df1["PMID"] == id_iter,"GitHub_link_raw"] = ", ".join(get_link(df1, id_iter))
    df1.loc[df1["PMID"] == id_iter,"GitHub_link_clean"] = ", ".join(clean_link(df1.loc[df1["PMID"] == id_iter,"GitHub_link_raw"]))
    df1.loc[df1["PMID"] == id_iter, "Phrase"] = str(get_phrase_with_link(df1, id_iter))

df1 = df1[["PMID", "PubDate", "DOI", "GitHub_link_raw", "GitHub_link_clean", "Journal", "Title", "Phrase", "Abstract"]]
pd.options.display.max_colwidth = None
df1

,PMID,PubDate,DOI,GitHub_link_raw,GitHub_link_clean,Journal,Title,Phrase,Abstract
0,36760999,2023-01-24,10.3389/fgene.2023.1082032,https://github.com/free1234hm/CLAM,https://github.com/free1234hm/CLAM,Frontiers in genetics,Identification of functional gene modules by integrating multi-omics data and known molecular interactions.,"We implemented Correlation-based Local Approximation of Membership as \na user-friendly application available at https://github.com/free1234hm/CLAM.\n\nCopyright © 2023 Chen, Han, Li, Li, Zhang and Zhu.","Front Genet. 2023 Jan 24;14:1082032. doi: 10.3389/fgene.2023.1082032. \neCollection 2023.\n\nIdentification of functional gene modules by integrating multi-omics data and \nknown molecular interactions.\n\nChen X(1)(2), Han M(2), Li Y(3), Li X(2), Zhang J(2), Zhu Y(1)(2).\n\nAuthor information:\n(1)Basic Medical School, Anhui Medical University, Hefei, China.\n(2)National Center for Protein Sciences (Beijing), Beijing Proteome Research \nCenter, Beijing Institute of Lifeomics, Beijing, China.\n(3)Central Research Laboratory, Peking Union Medical College Hospital, Chinese \nAcademy of Medical Sciences and Peking Union Medical College, Beijing, China.\n\nMulti-omics data integration has emerged as a promising approach to identify \npatient subgroups. However, in terms of grouping genes (or gene products) into \nco-expression modules, data integration methods suffer from two main drawbacks. \nFirst, most existing methods only consider genes or samples measured in all \ndifferent datasets. Second, known molecular interactions (e.g., transcriptional \nregulatory interactions, protein-protein interactions and biological pathways) \ncannot be utilized to assist in module detection. Herein, we present a novel \ndata integration framework, Correlation-based Local Approximation of Membership \n(CLAM), which provides two methodological innovations to address these \nlimitations: 1) constructing a trans-omics neighborhood matrix by integrating \nmulti-omics datasets and known molecular interactions, and 2) using a local \napproximation procedure to define gene modules from the matrix. Applying \nCorrelation-based Local Approximation of Membership to human colorectal cancer \n(CRC) and mouse B-cell differentiation multi-omics data obtained from The Cancer \nGenome Atlas (TCGA), Clinical Proteomics Tumor Analysis Consortium (CPTAC), Gene \nExpression Omnibus (GEO) and ProteomeXchange database, we demonstrated its \nsuperior ability to recover biologically relevant modules and gene ontology (GO) \nterms. Further investigation of the colorectal cancer modules revealed numerous \ntranscription factors and KEGG pathways that played crucial roles in colorectal \ncancer progression. Module-based survival analysis constructed four \nsurvival-related networks in which pairwise gene correlations were significantly \ncorrelated with colorectal cancer patient survival. Overall, the series of \nevaluations demonstrated the great potential of Correlation-based Local \nApproximation of Membership for identifying modular biomarkers for complex \ndiseases. We implemented Correlation-based Local Approximation of Membership as \na user-friendly application available at https://github.com/free1234hm/CLAM.\n\nCopyright © 2023 Chen, Han, Li, Li, Zhang and Zhu.\n\nDOI: 10.3389/fgene.2023.1082032\nPMCID: PMC9902936\nPMID: 36760999\n\nConflict of interest statement: The authors declare that the research was \nconducted in the absence of any commercial or financial relationships that could \nbe construed as a potential conflict of interest.\n\n"
1,36759692,2023-02-09,10.1038/s41598-023-29320-6,https://github.com/Socrates023/DCPHA,https://github.com/Socrates023/DCPHA,Scientific reports,Deep consistency-preserving hash auto-encoders for neuroimage cross-modal retrieval.,We make code and models publicly available: \nhttps://github.com/Socrates023/DCPHA .,"Sci Rep. 2023 Feb 9;13(1):2316. doi: 10.1038/s41598-023-29320-6.\n\nDeep consistency-pr

In [19]:
df1.to_csv('C:/Users/nadia/pubmed_results.tsv', sep='\t', index=False,columns=df1.columns[:-1].tolist())

In [ ]:
#https://github.com/free1234hm/CLAM

owner = "free1234hm"
repo = "CLAM"
url = f"https://api.github.com/repos/{owner}/{repo}"

response = requests.get(url)
if response.status_code == 200:
    repository_info = response.json()
    created_at = repository_info["created_at"]
    updated_at = repository_info["updated_at"]
created_at, updated_at   

('2022-10-27T15:54:53Z', '2022-10-31T19:16:00Z')

In [ ]:
#https://github.com/tyqGitHub/TYQ/tree/master/GACNNMDA - ????

In [ ]:
link = "https://github.com/free1234hm/CLAM"

owner = re.findall("\/[^\/.]+/", str(link))[0][1:-1]
repo = re.findall("\/[^\/.]+$", str(link))[0][1:-1]
url = f"https://api.github.com/repos/{owner}/{repo}"

response = requests.get(url)
if response.status_code == 200:
    repository_info = response.json()
    created_at = repository_info["created_at"]
    updated_at = repository_info["updated_at"]
else:
    created_at = None
    updated_at = None

df1.loc[df1["PMID"] == "36760999","Repo_created_at"], df1.loc[df1["PMID"] == "36760999","Repo_updated_at"] = created_at, updated_at

df1.loc[df1["PMID"] == "36760999","Repo_created_at"], df1.loc[df1["PMID"] == "36760999","Repo_updated_at"]

C:\Users\nadia\AppData\Local\Temp\ipykernel_12348\3653661987.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[df1["PMID"] == "36760999","Repo_created_at"], df1.loc[df1["PMID"] == "36760999","Repo_updated_at"] = created_at, updated_at
C:\Users\nadia\AppData\Local\Temp\ipykernel_12348\3653661987.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[df1["PMID"] == "36760999","Repo_created_at"], df1.loc[df1["PMID"] == "36760999","Repo_updated_at"] = created_at, updated_at


(0   NaN
 Name: Repo_created_at, dtype: float64,
 0   NaN
 Name: Repo_updated_at, dtype: float64)

In [ ]:
def get_repo_info(link):
    
    owner = re.findall("\/[^\/.]+/", str(link))[0][1:-1]
    repo = re.findall("\/[^\/.]+$", str(link))[0][1:-1]
    url = f"https://api.github.com/repos/{owner}/{repo}"
    
    response = requests.get(url)
    if response.status_code == 200:
        repository_info = response.json()
        created_at = repository_info["created_at"]
        updated_at = repository_info["updated_at"]
        return created_at, updated_at
    else:
        return None, None

In [ ]:
get_repo_info(df1.loc[df1["PMID"] == "36760999","GitHub_link_clean"])

(None, None)

In [ ]:
for id_iter in df1["PMID"]:
    
    df1.loc[df1["PMID"] == id_iter,"Repo_created_at"], df1.loc[df1["PMID"] == id_iter,"Repo_updated_at"] = get_repo_info(df1.loc[df1["PMID"] == id_iter,"GitHub_link_clean"])
df1

,PMID,PubDate,DOI,GitHub_link_raw,GitHub_link_clean,Journal,Title,Phrase,Abstract,Repo_created_at,Repo_updated_at
0,36760999,2023-01-24,10.3389/fgene.2023.1082032,https://github.com/free1234hm/CLAM,https://github.com/free1234hm/CLAM,Frontiers in genetics,Identification of functional gene modules by integrating multi-omics data and known molecular interactions.,"We implemented Correlation-based Local Approximation of Membership as \na user-friendly application available at https://github.com/free1234hm/CLAM.\n\nCopyright © 2023 Chen, Han, Li, Li, Zhang and Zhu.","Front Genet. 2023 Jan 24;14:1082032. doi: 10.3389/fgene.2023.1082032. \neCollection 2023.\n\nIdentification of functional gene modules by integrating multi-omics data and \nknown molecular interactions.\n\nChen X(1)(2), Han M(2), Li Y(3), Li X(2), Zhang J(2), Zhu Y(1)(2).\n\nAuthor information:\n(1)Basic Medical School, Anhui Medical University, Hefei, China.\n(2)National Center for Protein Sciences (Beijing), Beijing Proteome Research \nCenter, Beijing Institute of Lifeomics, Beijing, China.\n(3)Central Research Laboratory, Peking Union Medical College Hospital, Chinese \nAcademy of Medical Sciences and Peking Union Medical College, Beijing, China.\n\nMulti-omics data integration has emerged as a promising approach to identify \npatient subgroups. However, in terms of grouping genes (or gene products) into \nco-expression modules, data integration methods suffer from two main drawbacks. \nFirst, most existing methods only consider genes or samples measured in all \ndifferent datasets. Second, known molecular interactions (e.g., transcriptional \nregulatory interactions, protein-protein interactions and biological pathways) \ncannot be utilized to assist in module detection. Herein, we present a novel \ndata integration framework, Correlation-based Local Approximation of Membership \n(CLAM), which provides two methodological innovations to address these \nlimitations: 1) constructing a trans-omics neighborhood matrix by integrating \nmulti-omics datasets and known molecular interactions, and 2) using a local \napproximation procedure to define gene modules from the matrix. Applying \nCorrelation-based Local Approximation of Membership to human colorectal cancer \n(CRC) and mouse B-cell differentiation multi-omics data obtained from The Cancer \nGenome Atlas (TCGA), Clinical Proteomics Tumor Analysis Consortium (CPTAC), Gene \nExpression Omnibus (GEO) and ProteomeXchange database, we demonstrated its \nsuperior ability to recover biologically relevant modules and gene ontology (GO) \nterms. Further investigation of the colorectal cancer modules revealed numerous \ntranscription factors and KEGG pathways that played crucial roles in colorectal \ncancer progression. Module-based survival analysis constructed four \nsurvival-related networks in which pairwise gene correlations were significantly \ncorrelated with colorectal cancer patient survival. Overall, the series of \nevaluations demonstrated the great potential of Correlation-based Local \nApproximation of Membership for identifying modular biomarkers for complex \ndiseases. We implemented Correlation-based Local Approximation of Membership as \na user-friendly application available at https://github.com/free1234hm/CLAM.\n\nCopyright © 2023 Chen, Han, Li, Li, Zhang and Zhu.\n\nDOI: 10.3389/fgene.2023.1082032\nPMCID: PMC9902936\nPMID: 36760999\n\nConflict of interest statement: The authors declare that the research was \nconducted in the absence of any commercial or financial relationships that could \nbe construed as a potential conflict of interest.\n\n",NaN,NaN
1,36759692,2023-02-09,10.1038/s41598-023-29320-6,https://github.com/Socrates023/DCPHA,https://github.com/Socrates023/DCPHA,Scientific reports,Deep consistency-preserving hash auto-encoders for neuroimage cross-modal retrieval.,We make code and models publicly available: \nhttps://github.com/Socrates023/DCPHA .,"Sci Rep. 2023 Feb 9;13(1):2316. doi: 10.1038/s4